In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sklearn
import os
import PyPDF2

In [23]:
directory = "articles et bouquins"
corpus = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            with open("corpus.txt", "a", encoding="utf-8") as file2:
                file2.write(f"=== Contenu du fichier : {filename} ===\n")
                file2.write(text)
                file2.write("\n\n")

In [30]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Read the content of the corpus.txt file
with open("corpus.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Function to clean and preprocess text
def preprocess_text(text):
    text = re.sub(r' +', ' ', content)

    # Enlever les retours à la ligne successifs
    text = re.sub(r'\n+', '\n', content)

    # Supprimer les tirets au début de chaque phrase
    text = re.sub(r'^-', '', content, flags=re.MULTILINE)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english') + stopwords.words('french'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return ' '.join(words)

# Preprocess the content
cleaned_content = preprocess_text(content)

# Save the cleaned content back to the file
with open("corpus_cleaned.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_content)

print("Text preprocessing complete. Cleaned content saved to corpus_cleaned.txt.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Text preprocessing complete. Cleaned content saved to corpus_cleaned.txt.


In [3]:
import spacy
from spacy.matcher import Matcher

def Extract_Concepts(spacy_model, chapter, num_chapter):
    """
    :spacy_model : spacy model object
    :chapter : chapter -> str
    :num_chapter : number of the chapter in PMBOK -> int

    :return : list of concepts -> list
    """
    # doc object
    doc = spacy_model(chapter)
    # correction of POS
    for tok in doc:
        for i in range(7):
            if ( tok.pos_ != 'NUM' and tok.text.startswith(f'{num_chapter}.{i}')):
                tok.pos_ = 'NUM'

    # Matcher class object
    matcher = Matcher(nlp.vocab)
    #define the pattern
    Concepts_Pattern = [

    [
        {"POS": "NUM"},
        {"POS": "SPACE", "OP": "*"},
        {"POS": "NOUN", "OP": "+"},
        {"POS": "SPACE", "OP": "*"},
        {"POS": "ADJ", "OP": "+"},
        {"POS": "SPACE", "OP": "*"},
        {"POS": "NOUN", "OP": "+"}],
    [
        {"POS": "NUM"},
        {"POS": "ADJ", "OP": "+"},
        {"POS": "NOUN", "OP": "+"}],

    [
        {"POS": "NUM"},
        {"POS": "ADJ", "OP": "+"},
        {"POS": "CCONJ", "OP": "+"},
        {"POS": "NOUN", "OP": "+"},
        {"POS": "NOUN", "OP": "+"}],

    [
        {"POS": "NUM"},
        {"POS": "NOUN", "OP": "+"},],
    [
        {"POS": "NUM"},
        {"POS": "VERB"},
        {"POS": "NOUN"},],

    [
        {"POS": "NUM"},
        {"POS": "NOUN"},
        {"POS": "NOUN"},
        {"POS": "VERB"}],

          ]
    # add the Pattern matcher
    matcher.add("Concepts_Pattern",Concepts_Pattern)

    matches = matcher(doc)

    spans = [doc[start:end]  for _, start, end in matches if doc[start].text.startswith(f'{num_chapter}.')]

    Concepts_list = spacy.util.filter_spans(spans)

    return Concepts_list, doc



In [5]:
# Charger le modèle Spacy
nlp = spacy.load("en_core_web_sm")

# Lire le contenu du fichier texte
with open(r"C:\Users\Maxim\OneDrive\Bureau\M1 IA\M2\projetNLP\corpus_cleaned.txt", "r", encoding="utf-8") as file:
    content = file.read()

content = content.replace('\n','\n ')
content = content.replace('  ',' ')
content = content.lower()

In [6]:
# Remplacer "project_scope_management" par le contenu du fichier texte
# et définir le num_chapter approprié (dans cet exemple, j'ai utilisé 5)
nlp.max_length = len(content) + 1000  # Increase the max_length limit
scope_concepts, doc = Extract_Concepts(spacy_model=nlp, chapter=content, num_chapter=11)

# Supprimer les doublons
unique_scope_concepts = list(dict.fromkeys([concept.text for concept in scope_concepts]))

# Afficher les concepts uniques extraits
for concept in unique_scope_concepts:
    print(concept)


11.a square matrix
11.1 restarted fom
11.2 prescribed convergence
11.3 restarting technique
11.5 truncated fom
11.6 truncated q
11.7 parallel computing
11.9 numerical experiment
11.9.1 restarted gmres
11.9.2 restarted gmres
11.9.3 restarting ﬂation
11.9.4 restarting ﬂation augmentation
11.9.5 truncated q
11.10 historical note
11.2 show
11.3 study
11.7 and11.8 brieﬂy
11.3.3 residual vector
11.6095 sherman1
11.1 using notation
11.1using result
11.1 let gmres
11.4 consider matrix
11.3 generate
11.1 exact eigenpair matrix
11.1displays relative true residual norm
11.3displays result supg
11.3 supg
11.4shows residual norm problemdiff
11.5shows relative true residual norm function iteration number
11.6 display residual norm function computing time
11.4 diﬀ
11.5 raefsky1
11.6 raefsky1
11.7–11.8show residual norm
11.15 zoom
11.8 supg
11.10 supg
11.11 rajat27b
11.9.3 restarting deﬂation
11.12 rajat27b
11.13 matrix
11.18 increase mand pin way
11.14 matrix
11.15 matrix
11.19 show eigenvalue
11.2 c